<a href="https://colab.research.google.com/github/Mihirkool/JP-Morgan-Qualitative-analysis/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving Nat_Gas (1).csv to Nat_Gas (1).csv


In [3]:
import pandas as pd
from datetime import datetime

# Step 1: Load CSV
df = pd.read_csv("Nat_Gas (1).csv")
df['Dates'] = pd.to_datetime(df['Dates'])
price_lookup = dict(zip(df['Dates'], df['Prices']))

# Step 2: Define input parameters
injection_schedule = [
    (datetime(2020, 10, 31), 500000),
    (datetime(2020, 11, 30), 500000),
]

withdrawal_schedule = [
    (datetime(2020, 12, 31), 600000),
    (datetime(2021, 1, 31), 400000),
]

injection_rate_limit = 600000
withdrawal_rate_limit = 600000
max_storage_volume = 1000000
monthly_storage_cost = 100000
injection_cost_per_mmbtu = 0.01 * 1e6 / 1e6
withdrawal_cost_per_mmbtu = 0.01 * 1e6 / 1e6
transport_cost_each_time = 50000

# Step 3: Pricing function
def price_gas_contract(
    injection_schedule,
    withdrawal_schedule,
    injection_rate_limit,
    withdrawal_rate_limit,
    max_storage_volume,
    monthly_storage_cost,
    injection_cost_per_mmbtu,
    withdrawal_cost_per_mmbtu,
    transport_cost_each_time,
    price_lookup
):
    injection_schedule = sorted(injection_schedule)
    withdrawal_schedule = sorted(withdrawal_schedule)
    storage_balance = 0
    cashflow = 0
    months_with_storage = set()

    for date, volume in injection_schedule:
        if volume > injection_rate_limit:
            raise ValueError(f"Injection too high on {date}")
        if storage_balance + volume > max_storage_volume:
            raise ValueError(f"Too much gas in storage on {date}")
        price = price_lookup.get(date)
        if price is None:
            raise ValueError(f"No price for injection date {date}")
        purchase_cost = price * volume
        injection_cost = injection_cost_per_mmbtu * (volume / 1e6)
        cashflow -= (purchase_cost + injection_cost + transport_cost_each_time)
        storage_balance += volume
        months_with_storage.add(date.month + 12 * date.year)

    for date, volume in withdrawal_schedule:
        if volume > withdrawal_rate_limit:
            raise ValueError(f"Withdrawal too high on {date}")
        if volume > storage_balance:
            raise ValueError(f"Not enough gas to withdraw on {date}")
        price = price_lookup.get(date)
        if price is None:
            raise ValueError(f"No price for withdrawal date {date}")
        sale_revenue = price * volume
        withdrawal_cost = withdrawal_cost_per_mmbtu * (volume / 1e6)
        cashflow += (sale_revenue - withdrawal_cost - transport_cost_each_time)
        storage_balance -= volume
        months_with_storage.add(date.month + 12 * date.year)

    cashflow -= monthly_storage_cost * len(months_with_storage)
    return round(cashflow, 2)

# Step 4: Run the model
result = price_gas_contract(
    injection_schedule,
    withdrawal_schedule,
    injection_rate_limit,
    withdrawal_rate_limit,
    max_storage_volume,
    monthly_storage_cost,
    injection_cost_per_mmbtu,
    withdrawal_cost_per_mmbtu,
    transport_cost_each_time,
    price_lookup
)

print(f"Contract Value: ${result}")


Contract Value: $159999.98


/tmp/ipython-input-3-221816331.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])
